<a href="https://colab.research.google.com/github/KondaCharana/DataExtraction-NLP/blob/main/Probiotic_Data_Champion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Function to extract the title and main text from an HTML page
def extract_article_text(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)  # Add headers
        if response.status_code != 200:
            return None, f"Error: Received status code {response.status_code}"

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract title
        title_tag = soup.find('title')
        title = title_tag.get_text(strip=True) if title_tag else 'No Title'

        # Extract main content
        article_body = soup.find(['article', 'main'])  # Look for <article> or <main>
        if not article_body:
            article_body = soup.find('div', {'class': 'content'})  # Try a common div class
        if not article_body:
            paragraphs = soup.find_all('p')  # Fallback to paragraphs
            main_text = "\n".join([p.get_text(strip=True) for p in paragraphs])
        else:
            main_text = article_body.get_text("\n", strip=True)

        return title, main_text

    except requests.exceptions.RequestException as e:
        return None, str(e)

# Load the Excel file
input_file = 'Book1.xlsx'  # Ensure the file exists
data = pd.read_excel(input_file)

# Ensure the 'Website' column contains properly formatted URLs
def fix_url(url):
    if url.startswith('www.'):
        return 'https://' + url
    elif not url.startswith(('http://', 'https://')):
        return 'https://' + url  # Assume https if missing
    return url

data['Website'] = data['Website'].apply(fix_url)

# Ensure output directory exists
output_directory = 'Output'
os.makedirs(output_directory, exist_ok=True)

# Extract and save article text for each company
for index, row in data.iterrows():
    company_name = row['Company Name']
    url = row['Website']

    title, main_text = extract_article_text(url)

    if main_text and "Error" not in main_text:
        output_file_path = os.path.join(output_directory, f"{company_name.replace(' ', '_')}.txt")
        with open(output_file_path, 'w', encoding="utf-8") as f:
            f.write(f"Title: {title}\n\n{main_text}")

        print(f"Extracted data for {company_name} from URL: {url}")
    else:
        print(f"Failed to extract data for {company_name} from URL: {url}. Error: {main_text}")


Failed to extract data for Nestle from URL: https://www.nestle.com. Error: Error: Received status code 403
Failed to extract data for Dr. Reddy's Laboratories from URL: https://www.drreddys.com. Error: 
Extracted data for Coca from URL: https://colacompany.com
Extracted data for Pfizer from URL: https://www.pfizer.com
Extracted data for PepsiCo from URL: https://www.pepsico.com
Extracted data for Johnson & Johnson from URL: https://www.jnj.com
Extracted data for Danone from URL: https://www.danone.com
Failed to extract data for Bayer from URL: https://www.bayer.com. Error: Error: Received status code 403
Extracted data for General Mills from URL: https://www.generalmills.com
Extracted data for GlaxoSmithKline (GSK) from URL: https://www.gsk.com
Extracted data for Kellogg’s from URL: https://www.kelloggs.com
Extracted data for Merck & Co. from URL: https://www.merck.com
Extracted data for Unilever from URL: https://www.unilever.com
Extracted data for Roche from URL: https://www.roche.co

In [52]:
import pandas as pd
import os

# Define your keywords
keywords = {
    "F&B": ["food", "beverage", "nutrition", "edible", "fortified", "grocery", "milk", "cereal", "pasta", "bakery", "juice",
            "probiotic", "health drink", "energy drink", "dairy", "confectionery", "fermented", "snacks", "gut health"],

    "Manufacturer": ["manufacture", "production", "factory", "processing", "ISO", "GMP", "FDA-approved", "bulk processing",
                     "dosage form", "contract manufacturer", "plant certification", "ISO 9001", "USP standard", "formulations"],

    "Brand": ["brand", "product line", "trademark", "buy", "purchase", "retail", "e-commerce", "pricing", "gut health",
              "women's health", "sports nutrition", "mental wellness", "cognitive health", "pharmacy brand"],

    "Distributor": ["wholesaler", "reseller", "supplier", "import", "export", "logistics", "supply chain", "procurement",
                    "B2B supplier", "bulk raw materials", "nutraceutical distributor", "pharma distributor"],

    "Bulk": ["pharma", "nutraceutical", "raw material", "strain", "bacteria", "bacillus coagulans", "saccharomyces",
             "bulk customer", "ingredient supplier", "probiotic blends", "contract manufacturing"],

    "Health Segment": ["gut health", "women's health", "cognitive health", "mental wellness", "sports nutrition",
                       "digestive health", "PCOD", "UTI", "Bacipro", "Provinorm", "Cognisol"]
}

# Load your Excel file
excel_file = "input.xlsx"  # Replace with your Excel file name
df = pd.read_excel(excel_file)

# Add new columns and initialize with "no"
for category in keywords:
    df[category] = "no"

# Process each company
'''
for index, row in df.iterrows():
    company_name = row['Company Name']
    text_file_path = os.path.join("Output", f"{company_name}.txt")  # Assuming text files are in "Output" folder

    try:
        with open(text_file_path, 'r') as f:
            text = f.read().lower()  # Read and convert to lowercase for case-insensitive matching

            # Check for keywords and update Excel columns
            for category, keyword_list in keywords.items():
                if any(keyword in text for keyword in keyword_list):
                    df.loc[index, category] = "yes"
    except FileNotFoundError:
        print(f"Text file not found for {company_name}")
'''
for index, row in df.iterrows():
    company_name = row['Company Name']
    text_file_path = os.path.join("Output", f"{company_name}.txt")  # Assuming text files are in "Output" folder

    try:
        with open(text_file_path, 'r') as f:
            text = f.read().lower()  # Read and convert to lowercase for case-insensitive matching

            found = False  # Track if any keyword is found
            for category, keyword_list in keywords.items():
                if any(keyword in text for keyword in keyword_list):
                    df.loc[index, category] = "Yes"
                    found = True

            # If no keyword matched, mark all categories as "Not Relevant"
            if not found:
                for category in keywords:
                    df.loc[index, category] = "Not Relevant"
    except FileNotFoundError:
        print(f"Text file not found for {company_name}")
# Save the updated Excel file
df.to_excel("output.xlsx", index=False)  # Replace with desired output file name
print("Keywords checked and Excel file updated successfully.")

Text file not found for Dr. Reddy's Laboratories
Text file not found for Boehringer Ingelheim
Keywords checked and Excel file updated successfully.
